In [1]:
from langchain_community.agent_toolkits import GmailToolkit

toolkit = GmailToolkit()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=376714772999-edi72200ielsailvih1csvre9vmmmmmf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51766%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=4xVX5utLNZsKcLQFzy95652CnMkPv2&access_type=offline


In [2]:
from langchain_community.tools.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
toolkit = GmailToolkit(api_resource=api_resource)

In [3]:
tools = toolkit.get_tools()
tools

[GmailCreateDraft(api_resource=<googleapiclient.discovery.Resource object at 0x000001FDCF6966B0>),
 GmailSendMessage(api_resource=<googleapiclient.discovery.Resource object at 0x000001FDCF6966B0>),
 GmailSearch(api_resource=<googleapiclient.discovery.Resource object at 0x000001FDCF6966B0>),
 GmailGetMessage(api_resource=<googleapiclient.discovery.Resource object at 0x000001FDCF6966B0>),
 GmailGetThread(api_resource=<googleapiclient.discovery.Resource object at 0x000001FDCF6966B0>)]

In [10]:
import getpass
import os

api_key = os.getenv("OPENAI_API_KEY")

In [5]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

In [6]:
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)


In [11]:
llm = ChatOpenAI(model="gpt-4-turbo",temperature=0,openai_api_key=api_key)
agent = create_openai_functions_agent(llm, toolkit.get_tools(), prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    # This is set to False to prevent information about my email showing up on the screen
    # Normally, it is helpful to have it set to True however.
    verbose=False,
)


In [12]:
agent_executor.invoke(
    {
        "input": "show me the latest 5 incoming emails."
    }
)

{'input': 'show me the latest 5 incoming emails.',
 'output': "Here are the latest 5 incoming emails in your inbox:\n\n1. **From:** Qatar Airways Privilege Club <email@qr.qatarairways.com>\n   **Subject:** Dhanashekar, drive your way to triple Avios with Avis\n   **Snippet:** Hire a vehicle to unlock amazing rewards. Drive your way to triple Avios with Avis and enjoy rewards on your car rentals.\n\n2. **From:** Target <targetnews@em.target.com>\n   **Subject:** Inside: your weekly Target Circle deals 🎯\n   **Snippet:** The easy way to find all your savings for the week. Check out the latest deals and new arrivals at Target.\n\n3. **From:** Tripadvisor <inspiration@mp1.tripadvisor.com>\n   **Subject:** 12 supremely artsy cities we love 🎨\n   **Snippet:** Plus, a relaxing weekend in Hawaii. Explore travel tips, insights, and inspiration for your next adventure.\n\n4. **From:** TechCareers.com <alert@email.nexxt.com>\n   **Subject:** Alert: Following Research Intern\n   **Snippet:** New J